In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fed-non-iid-dataset/DDoS_client_1.csv
/kaggle/input/fed-non-iid-dataset/Spoofing_client_3.csv
/kaggle/input/fed-non-iid-dataset/WebBased_client_2.csv
/kaggle/input/fed-non-iid-dataset/Recon_client_4.csv
/kaggle/input/fed-non-iid-dataset/Dos_client_5.csv


In [2]:
!pip install -U imbalanced-learn

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import load_model, Sequential
from keras.layers import Dense, Activation
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report
import seaborn as sns
%matplotlib inline
from sklearn.utils import shuffle

2024-06-07 16:49:59.812201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 16:49:59.812346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 16:49:59.996657: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
ddos = pd.read_csv('/kaggle/input/fed-non-iid-dataset/DDoS_client_1.csv')
dos = pd.read_csv('/kaggle/input/fed-non-iid-dataset/Dos_client_5.csv')
webbased = pd.read_csv('/kaggle/input/fed-non-iid-dataset/WebBased_client_2.csv')

In [5]:
dos.label.value_counts()

label
1    210944
0     23355
Name: count, dtype: int64

In [6]:
ddos = shuffle(ddos).reset_index(drop=True)
dos = shuffle(dos).reset_index(drop=True)
webbased = shuffle(webbased).reset_index(drop=True)

In [7]:
X = ddos.iloc[:, :-1]
y = ddos.iloc[:, -1]
x_train_ddos, x_test_ddos, y_train_ddos, y_test_ddos = train_test_split(X, y, test_size=0.1, random_state=42)

scalar = StandardScaler()
scalar.fit(x_train_ddos)

x_train_ddos = scalar.transform(x_train_ddos)
x_test_ddos = scalar.transform(x_test_ddos)


In [8]:
X = dos.iloc[:, :-1]
y = dos.iloc[:, -1]
x_train_dos, x_test_dos, y_train_dos, y_test_dos = train_test_split(X, y, test_size=0.1, random_state=42)

scalar = StandardScaler()
scalar.fit(x_train_dos)

print(y_train_dos.value_counts())
x_train_dos = scalar.transform(x_train_dos)
x_test_dos = scalar.transform(x_test_dos)


from imblearn.over_sampling import SMOTE

# Assume X_train and y_train are your data
sm = SMOTE(sampling_strategy={0: 189872}, random_state=42)
X_res, y_res = sm.fit_resample(x_train_dos, y_train_dos)

label
1    189862
0     21007
Name: count, dtype: int64


In [9]:
X_res.shape

(379734, 33)

In [10]:
X_ = webbased.iloc[:, :-1]
y_ = webbased.iloc[:, -1]
x_train_webbased, x_test_webbased, y_train_webbased, y_test_webbased = train_test_split(X, y, test_size=0.1, random_state=42)

scalar = StandardScaler()
# scalar.fit(x_train_webbased)

# x_train_webbased = scalar.transform(x_train_webbased)
# x_test_webbased = scalar.transform(x_test_webbased)

scalar.fit(X_)

X_ = scalar.transform(X_)

### Concatenating the all the test data.

In [11]:
X_test_all = np.concatenate([x_test_ddos, x_test_dos], axis=0)
y_test_all = np.concatenate([y_test_ddos, y_test_dos], axis=0)

print(X_test_all.shape)
print(y_test_all.shape)

(75378, 33)
(75378,)


### Defining the model

In [12]:
# initializing the model
model = Sequential()
model.add(tf.keras.layers.Input(shape=(33,)))

model.add(Dense(units=4))
model.add(Activation('relu'))

model.add(Dense(units=4))
model.add(Activation('relu'))

model.add(Dense(units=1),)
model.add(Activation('sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [13]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
history1 = model.fit(X_res, y_res, epochs=40, verbose=1, validation_split=0.1, batch_size=128, callbacks=[callback])

Epoch 1/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.6942 - loss: 0.6622 - val_accuracy: 0.7994 - val_loss: 0.5843
Epoch 2/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7520 - loss: 0.6285 - val_accuracy: 0.8378 - val_loss: 0.5357
Epoch 3/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7718 - loss: 0.5879 - val_accuracy: 0.8568 - val_loss: 0.4832
Epoch 4/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7799 - loss: 0.5436 - val_accuracy: 0.8714 - val_loss: 0.4304
Epoch 5/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8823 - loss: 0.4896 - val_accuracy: 0.8838 - val_loss: 0.3816
Epoch 6/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9340 - loss: 0.4336 - val_accuracy: 0.8942 - val_loss: 0.3374
Epoch 7/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9408 - loss: 0.3801 - val_accuracy: 0.9030 - val_loss: 0.2985
Epoch 8/40
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9448 - loss: 0.3307 - 

### Testing the model on a test set from the same distribution

In [14]:
y_pred = model.predict(x_test_dos)
y_pred = (y_pred > 0.5)
print('\n\nConfusion Matrixs')
print(confusion_matrix(y_test_dos, y_pred))
print('\n\nEvaluation metrics')
print(classification_report(y_test_dos, y_pred, digits=5))

733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  


Confusion Matrixs
[[ 2325    23]
 [  161 20921]]


Evaluation metrics
              precision    recall  f1-score   support

           0    0.93524   0.99020   0.96194      2348
           1    0.99890   0.99236   0.99562     21082

    accuracy                        0.99215     23430
   macro avg    0.96707   0.99128   0.97878     23430
weighted avg    0.99252   0.99215   0.99225     23430



### Evaluating the model as it meets different attacks

In [15]:
y_pred = model.predict(X_test_all)
y_pred = (y_pred > 0.5)
print('\n\nConfusion Matrixs')
print(confusion_matrix(y_test_all, y_pred))
print('\n\nEvaluation metrics')
print(classification_report(y_test_all, y_pred, digits=5))

2356/2356 ━━━━━━━━━━━━━━━━━━━━ 2s 959us/step


Confusion Matrixs
[[ 3899    61]
 [ 7239 64179]]


Evaluation metrics
              precision    recall  f1-score   support

           0    0.35006   0.98460   0.51649      3960
           1    0.99905   0.89864   0.94619     71418

    accuracy                        0.90315     75378
   macro avg    0.67456   0.94162   0.73134     75378
weighted avg    0.96496   0.90315   0.92361     75378



### Evaluating the model when it's being used in entirely different environment

In [16]:
y_pred = model.predict(x_test_webbased)
y_pred = (y_pred > 0.5)
print('\n\nConfusion Matrixs')
print(confusion_matrix(y_test_webbased, y_pred))
print('\n\nEvaluation metrics')
print(classification_report(y_test_webbased, y_pred, digits=5))

733/733 ━━━━━━━━━━━━━━━━━━━━ 1s 993us/step


Confusion Matrixs
[[   12  2336]
 [    5 21077]]


Evaluation metrics
              precision    recall  f1-score   support

           0    0.70588   0.00511   0.01015      2348
           1    0.90023   0.99976   0.94739     21082

    accuracy                        0.90009     23430
   macro avg    0.80305   0.50244   0.47877     23430
weighted avg    0.88075   0.90009   0.85346     23430



In [17]:
y_pred = model.predict(X_)
y_pred = (y_pred > 0.5)
print('\n\nConfusion Matrixs')
print(confusion_matrix(y_, y_pred))
print('\n\nEvaluation metrics')
print(classification_report(y_, y_pred, digits=5))

1073/1073 ━━━━━━━━━━━━━━━━━━━━ 1s 974us/step


Confusion Matrixs
[[11035 22520]
 [  214   537]]


Evaluation metrics
              precision    recall  f1-score   support

           0    0.98098   0.32886   0.49259     33555
           1    0.02329   0.71505   0.04511       751

    accuracy                        0.33732     34306
   macro avg    0.50213   0.52195   0.26885     34306
weighted avg    0.96001   0.33732   0.48279     34306

